In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [21]:
url_query = "https://www.sewakost.com/jakarta/selatan/"
req = requests.get(url_query, verify=False)
soup = BeautifulSoup(req.content, "html.parser")
str(soup)[:100]

'<!DOCTYPE html>\n\n<html lang="id" xmlns="http://www.w3.org/1999/xhtml">\n<head>\n<title>Kost Jakarta Se'

In [3]:
try:
    find_pagination = soup.find("ul", attrs = {"class": "pagination"})
    find_span = find_pagination.findAll("span")
    total_page = int(find_span[1].text[-2:])
    print("Total Page: {}".format(total_page))
except:
    print("Article Not Found")

Total Page: 89


In [11]:
sewakost = []

for page_num in tqdm(range(1, total_page+1)[:40]):
    url_sewakost = "https://www.sewakost.com/jakarta/selatan/index%s.html" % page_num
    # print(url_sewakost)
    r = requests.get(url_sewakost, verify=False)
    s = BeautifulSoup(r.content, "html.parser")
    
    find_kos = s.findAll("div", attrs = {"class": "main-column clearfix"})
    for row in find_kos:
        try:
          kos = {}
          kos['url'] = row.ul.li.a['href']
          kos['nama'] = row.ul.li.a['title']
          harga = row.find("span", attrs = {"class": "price-tag"})
          kos['harga'] = harga.span.text
          sewakost.append(kos)
        except:
          continue
        

100%|██████████| 40/40 [00:58<00:00,  1.46s/it]


In [12]:
pd.DataFrame(sewakost)

,url,nama,harga
0,https://www.sewakost.com/jakarta/selatan/kost-...,Kost Ciganjur Jalan Pasir No. 32,Rp 1.000.000
1,https://www.sewakost.com/jakarta/selatan/kost-...,KOST EXECUTIVE JAGAKARSA PASO CILANDAK JAKARTA...,Rp 2.200.000
2,https://www.sewakost.com/jakarta/selatan/kamar...,Kamar kost Putri NON AC di Mampang,Rp 850.000
3,https://www.sewakost.com/jakarta/selatan/hompy...,Hompy Timur Residence,Rp 2.700.000
4,https://www.sewakost.com/jakarta/selatan/rumah...,Rumah kontrakan (40m2) Pasutri baru/karyawati/...,Rp 2.500.000
...,...,...,...
953,https://www.sewakost.com/jakarta/selatan/kost-...,Kost lokasi Strategis dan Aman,Rp 2.200.000
954,https://www.sewakost.com/jakarta/selatan/kosan...,Kosan 28 Duren Tiga (Bangunan Baru!!),Rp 1.100.000
955,https://www.sewakost.com/jakarta/selatan/kost-...,"Kost Khusus Putra di Tebet Timur Dalam 7G, Jak...",Rp 1.900.000
956,https://www.sewakost.com/jakarta/selatan/kost-...,Kost Putri Qtha,Rp 1.400.000


In [24]:
for kos in tqdm(sewakost):
  try:
    req_kos = requests.get(kos['url'], verify = False)
    soup_kos = BeautifulSoup(req_kos.content, "html.parser")

    informasi = ['jenis', 'address', 'aircon', 'free_wifi', 'kamar_mandi_dalam']

    for info in informasi:
      value = soup_kos.find("div", attrs={"id": "df_field_%s" % info}).find("div", attrs={"class":"value"})
      kos[info] = value.text.strip()

    kos['fasilitas_kamar'] = []
    kos['fasilitas_kost'] = []
    fasilitas = ['fasilitas_kamar', 'fasilitas_kost']

    for f in fasilitas:
      list_f = soup_kos.find("div", attrs={"id": "df_field_%s" % f}).find("div", attrs={"class":"value"})
      for each_f in list_f.findAll("li"):
        kos[f].append(each_f['title'])

  except:
    pass

100%|██████████| 958/958 [23:26<00:00,  1.47s/it]


Save

In [26]:
pd.DataFrame(sewakost).to_csv('sewakost_40page.tsv', sep="\t", index=False)

In [29]:
df = pd.read_csv('sewakost_40page.tsv', sep="\t")

df.head()

,url,nama,harga,jenis,address,aircon,free_wifi,kamar_mandi_dalam,fasilitas_kamar,fasilitas_bersama,fasilitas_kost
0,https://www.sewakost.com/jakarta/selatan/kost-...,Kost Ciganjur Jalan Pasir No. 32,Rp 1.000.000,Kost Campur,Jalan Pasir no 32 Kp. Setu Rt 001/006 Keluraha...,Tidak,Tidak,Ya,['Kamar Mandi Dalam'],"['Keamanan', 'Parkir Motor']","['Keamanan', 'Parkir Motor']"
1,https://www.sewakost.com/jakarta/selatan/kost-...,KOST EXECUTIVE JAGAKARSA PASO CILANDAK JAKARTA...,Rp 2.200.000,Kost Putra,"Jalan Paso no 6A, Jagakarsa, Cilandak, Jakarta...",Ya,Ya,Ya,"['Spring Bed', 'Perabot', 'Free WiFi', 'AC', '...","['Ruang Tamu', 'Ruang Makan', 'Dapur', 'Air Mi...","['Ruang Tamu', 'Ruang Makan', 'Dapur', 'Air Mi..."
2,https://www.sewakost.com/jakarta/selatan/kamar...,Kamar kost Putri NON AC di Mampang,Rp 850.000,Kost Putri,Jl.Mampang Prapatan VI Rt04 Rw02 No.70B,Tidak,Tidak,Tidak,[],NaN,[]
3,https://www.sewakost.com/jakarta/selatan/hompy...,Hompy Timur Residence,Rp 2.700.000,Kost Putri,Tebet Timur Dalam 7B no. 11,Ya,Ya,Ya,"['Spring Bed', 'Perabot', 'Free WiFi', 'AC', '...","['Dapur', 'Cuci Setrika', 'Pembantu', 'Keamana...","['Dapur', 'Cuci Setrika', 'Pembantu', 'Keamana..."
4,https://www.sewakost.com/jakarta/selatan/rumah...,Rumah kontrakan (40m2) Pasutri baru/karyawati/...,Rp 2.500.000,Kost Pasutri,Jalan keselamatan 18 RT10/RW03,Ya,Tidak,Ya,"['AC', 'Kamar Mandi Dalam']","['Ruang Tamu', 'Ruang Makan', 'Dapur', 'Keaman...","['Ruang Tamu', 'Ruang Makan', 'Dapur', 'Keaman..."
